In [1]:
from langchain_ollama import OllamaLLM
import pandas as pd
import ast
import re
from difflib import SequenceMatcher

def askOllama(prompt):
    result = model.invoke(input=prompt)
    return result

def cleanOllamaOutput(output):
    pattern = r"\[.*?\]"
    matches = re.findall(pattern, output, re.DOTALL)
    a = matches[0].replace("\n", "")
    return ast.literal_eval(a)

model = OllamaLLM(model="llama3.2")

annotatedDataset = pd.read_csv("/Users/sagewong/git/StigmatizingLanguageProject/FinalFinalAnnnotatedData.csv")


In [14]:
true_positive = 0
false_positive = 0
true_negative = 0
false_negative = 0
for index in range(annotatedDataset.shape[0]):
    print("INDEX: " + str(index))
    clinicalNote = annotatedDataset.iloc[index]['Completion']

    answer = ast.literal_eval(annotatedDataset.iloc[index]['annotated'])


    prompt = "You are a professional linguist researcher who is trying to identify stigmatizing language in clinical notes. Given this clinical note, return to me in a python-type list all forms of stigmatizing language (e.g. noncompliant, nonadherent, etc...). Do not include any descriptions or explanations. Also do not rewrite the stigmatizing language in your own words. Here's the note: " + clinicalNote + "Make sure to output in JSON format."

    rawOutput = askOllama(prompt)
    cleanedOutput = cleanOllamaOutput(rawOutput)
    print(cleanedOutput)
    print(answer)

    matching = []
    for i in cleanedOutput:
        for j in answer:
            if i in j:
                matching.append(i)

    print(matching)

    true_positive += len(matching)
    false_positive += len([i for i in cleanedOutput if i not in matching])
    true_negative += clinicalNote.count(" ")/3 - len(matching) + len([i for i in answer if answer not in matching])*3
    false_negative += len([i for i in answer if answer not in matching])

    tempTruePositive = len(matching)
    tempFalsePositive = len([i for i in cleanedOutput if i not in matching])
    tempTrueNegative = clinicalNote.count(" ")/3 - len(matching) + len([i for i in answer if answer not in matching])*3
    tempFalseNegative = len([i for i in answer if answer not in matching])
    tempPrecision = tempTruePositive/(tempTruePositive + tempFalsePositive)
    tempRecall = tempTruePositive/(tempTruePositive + tempFalseNegative)
    try:    
        tempf1score = (2*tempPrecision*tempRecall)/(tempPrecision+tempRecall)
        print("SINGLE NOTE F1 SCORE: " + str(tempf1score))
    except:
        print("SINGLE NOTE F1 SCORE: 0")
    precision = true_positive/(true_positive + false_positive)
    recall = true_positive/(true_positive + false_negative)

    f1_score = (2*precision*recall)/(precision+recall)
    print("TOTAL F1 SCORE: " + str(f1_score))

precision = true_positive/(true_positive + false_positive)
recall = true_positive/(true_positive + false_negative)

f1_score = (2*precision*recall)/(precision+recall)
print(precision)
print(recall)
print(f1_score)

INDEX: 0
['noncompliant', 'nonadherent', 'uncooperative behavior', 'cursing at nurses', 'challenging interactions', 'reliable historical recounting', 'refused exams', 'resistant to modifying diet']
['challenging patient', 'uncooperative behavior', 'noncompliant', 'citing his preferences for junk food', 'frequent cursing at nurses', 'difficult patient', 'challenging interactions', 'unreliable historical recounting', 'refused several exams', 'resistant to modifying his diet', 'complicating our attempts to provide care', 'significant barrier to his care', 'noncompliance has been a significant barrier to his care', "Mohammad's noncompliance has been a significant barrier to his care"]
['noncompliant', 'uncooperative behavior', 'cursing at nurses', 'challenging interactions', 'reliable historical recounting']
SINGLE NOTE F1 SCORE: 0.37037037037037035
TOTAL F1 SCORE: 0.37037037037037035
INDEX: 1
['noncompliant', 'nonadherent', 'uncooperative']
['uncooperative with blood draws and other proce